# VTK - ITK Project : Etude longitudinale de l'évolution d'une tumeur

### Par Raphaël Duhen, Maël Conan et Nigel Andrews

In [23]:
import vtk
import itk
import matplotlib.pyplot as plt
import numpy as np

import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display, clear_output

In [24]:
def run_window(path: str):
    reader = vtk.vtkNrrdReader()
    reader.SetFileName(path)
    
    window = vtk.vtkRenderWindow()
    renderer = vtk.vtkRenderer()
    
    window.AddRenderer(renderer)
    
    interactor = vtk.vtkRenderWindowInteractor()
    window.SetInteractor(interactor)
    
    contour = vtk.vtkContourFilter()
    contour.SetInputConnection(reader.GetOutputPort())
    contour.SetValue(0, 135)
    contour.SetFastOff()
    
    contourMapper = vtk.vtkPolyDataMapper()
    contourMapper.SetInputConnection(contour.GetOutputPort())
    contourMapper.ScalarVisibilityOff()
    
    contourActor = vtk.vtkActor()
    contourActor.SetMapper(contourMapper)
    
    renderer.AddActor(contourActor)
    
    window.Render()
    interactor.Start()

In [25]:
PixelType = itk.D
Dimension = 3
ImageType = itk.Image[PixelType, Dimension]
TransformType = itk.TranslationTransform[PixelType, Dimension]

In [26]:
image = itk.imread("Data/case6_gre1.nrrd", pixel_type=PixelType)
image2 = itk.imread("Data/case6_gre2.nrrd", pixel_type=PixelType) 

In [34]:
def registrate(image, image2):
    dimension = image.GetImageDimension()
    TransformType = itk.TranslationTransform[itk.D, dimension]

    initial_transform = TransformType.New()
    initial_transform.SetIdentity()

    optimizer = itk.RegularStepGradientDescentOptimizerv4.New(
        LearningRate=4,
        MinimumStepLength=0.001,
        RelaxationFactor=0.5,
        NumberOfIterations=200,
    )

    metric = itk.MeanSquaresImageToImageMetricv4[ImageType, ImageType].New()

    registration = itk.ImageRegistrationMethodv4[ImageType, ImageType].New(
        Metric=metric,
        Optimizer=optimizer,
        InitialTransform=initial_transform,
        FixedImage=image,
        MovingImage=image2,
    )

    registration.Update()

    return registration, optimizer

In [28]:
def transform_image(image, image2, registration):
    transformed_image = itk.resample_image_filter(
        image2,
        transform=registration.GetTransform(),
        use_reference_image=True,
        reference_image=image,
        default_pixel_value=100,
    )

    return transformed_image

In [39]:
registration = None

try:
    transformed_image = itk.imread("Data/transformed_image.nrrd", pixel_type=PixelType)
except:
    registration, optimizer = registrate(image, image2)
    transformed_image = transform_image(image, image2, registration)

In [36]:
clear_output()

array_view = itk.array_view_from_image(transformed_image)
original_fixed_view = itk.array_view_from_image(image)
original_moving_view = itk.array_view_from_image(image2)

@interact(fixed=(0, original_fixed_view.shape[0]),
          moving=(0, original_moving_view.shape[0]),
          transformed=(0, array_view.shape[0]))
def plot_slices(fixed: int, moving: int, transformed: int):
    plt.figure(figsize=(15, 15))
    plt.subplot(1, 3, 1)
    plt.imshow(original_fixed_view[fixed, :, :], cmap="gray")
    plt.title("Fixed image")
    plt.subplot(1, 3, 2)
    plt.imshow(original_moving_view[moving, :, :], cmap="gray")
    plt.title("Moving image")
    plt.subplot(1, 3, 3)
    plt.imshow(array_view[transformed, :, :], cmap="gray")
    plt.title("Transformed image")
    plt.show()

interactive(children=(IntSlider(value=88, description='fixed', max=176), IntSlider(value=88, description='movi…

In [37]:
itk.imwrite(transformed_image, "Data/transformed_image.nrrd")

In [41]:
if registration is not None:
    final_parameters = registration.GetOutput().Get().GetParameters()

    final_number_of_iterations = optimizer.GetCurrentIteration()

    print("Final parameters = ", final_parameters.GetElement(0))
    print("Final parameters = ", final_parameters.GetElement(1))
    print("Final parameters = ", final_parameters.GetElement(2))
    print("Final parameters = ", final_parameters.GetElement(3))
    print("Final parameters = ", final_parameters.GetElement(4))
    print("Final parameters = ", final_parameters.GetElement(5))

    print("Number of iterations = ", final_number_of_iterations)

In [42]:
def segmentation(image):
    PixelType = itk.D
    Dimension = 3
    ImageType = itk.Image[PixelType, Dimension]

    segmentation_filter = itk.ConnectedThresholdImageFilter[ImageType, ImageType].New(
        Lower=100,
        Upper=200,
        ReplaceValue=255,
        Seed=itk.Index[Dimension]([100, 100, 100]),
    )
    segmentation_filter.SetInput(image)
    segmentation_filter.Update()

    return segmentation_filter.GetOutput()

In [43]:
segmented_image = segmentation(transformed_image)

In [44]:
segmented_slices = itk.array_view_from_image(segmented_image)

In [45]:
clear_output()

@interact(slice=(0, segmented_slices.shape[0] - 1))
def plot_segmentation(slice: int):
    plt.figure(figsize=(15, 15))
    plt.subplot(1, 2, 1)
    plt.imshow(array_view[slice, :, :], cmap="gray")
    plt.title("Transformed image")
    plt.subplot(1, 2, 2)
    plt.imshow(segmented_slices[slice, :, :], cmap="gray")
    plt.title("Segmented image")
    plt.show()

interactive(children=(IntSlider(value=87, description='slice', max=175), Output()), _dom_classes=('widget-inte…